In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow.contrib.slim as slim


mnist = input_data.read_data_sets('/tmp/mnist/data',one_hot=True)

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 28*28], name='input')
y_actual = tf.placeholder(tf.float32, shape=[None, 10], name='label')
is_training = tf.placeholder(tf.bool, name='is_training')

def convnet(inputs, is_training, scope='Convnet'):
    with tf.variable_scope(scope, 'Convnet'):
        inputs= tf.reshape(inputs, [-1,28,28,1])
        # First Group: Convolution + Pooling 28x28x1 => 28x28x20 => 14x14x20
        net = slim.conv2d(inputs, 20, [5, 5], padding='SAME', scope='layer1-conv')
        net = slim.max_pool2d(net, 2, stride=2, scope='layer2-max-pool')

        # Second Group: Convolution + Pooling 14x14x20 => 10x10x40 => 5x5x40
        net = slim.conv2d(net, 40, [5, 5], padding='VALID', scope='layer3-conv')
        net = slim.max_pool2d(net, 2, stride=2, scope='layer4-max-pool')

        # Reshape: 5x5x40 => 1000x1
        net = slim.flatten(net)

        # Fully Connected Layer: 1000x1 => 1000x1
        net = slim.fully_connected(net, 1000, scope='layer5')
        net = slim.dropout(net,keep_prob=0.5, is_training=is_training, scope='layer5-dropout')

        # Second Fully Connected: 1000x1 => 1000x1
        net = slim.fully_connected(net, 1000, scope='layer6')
        net = slim.dropout(net,keep_prob=0.5, is_training=is_training, scope='layer6-dropout')

        # Output Layer: 1000x1 => 10x1
        net = slim.fully_connected(net, 10, scope='output')
        net = slim.dropout(net,keep_prob=0.5, is_training=is_training, scope='output-dropout')

        return net

logits = convnet(x, is_training, scope='Covnet')

with tf.name_scope('Loss'):
	cross_entropy =  slim.losses.softmax_cross_entropy(logits, y_actual)
with tf.name_scope('Prediction'):
	correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_actual, 1))
	accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
with tf.name_scope('Optimizer'):
    train_step = tf.train.MomentumOptimizer(0.03,0.5).minimize(cross_entropy)

sess.run(tf.global_variables_initializer())

val_data = {
    x: mnist.validation.images,
    y_actual: mnist.validation.labels,
    is_training: False
}

for i in range(2500):
    images, labels = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: images, y_actual: labels, is_training: True})
    
    if i % 100 == 0:
        acc = sess.run( accuracy, feed_dict= val_data)
        print("Step: %5d,CV Accuracy = %5.2f%%" % (i, acc * 100))

test_data = {
    x: mnist.test.images,
    y_actual: mnist.test.labels,
    is_training: False
}

acc = sess.run(accuracy, feed_dict= test_data)

print("Test Accuracy is %5.2f%%" % (100 * acc))